In [1]:
import datetime
import time

import requests
import pandas as pd

In [2]:
def get_checkin_data(lat, long):

#facebook API Authentification/public identifier
    client_id = "306273076895372"
    client_secret = "65d11b319633406f9ec1e772158efd4c"
    access_token = "{}|{}".format(client_id,client_secret)

    url = 'https://graph.facebook.com/search'
    params = {
        'type':'place',
        'fields':'name,checkins,engagement,overall_star_rating, single_line_address,website',
        'center':'{},{}'.format(lat, long),
        'distance':1000,
        'access_token':access_token,
        'limit':100   #cities - since there's ~20,000 US cities, make it 500? More?  
    }
    r = requests.get(url, params=params)
    data = r.json()['data']


    df = pd.DataFrame(data).sort_values('checkins', ascending = False)
    df['engagement'] = df['engagement'].apply(lambda x: x['count'])
    return df 

In [3]:
def get_checkins_around_latlong(lat,long):
    df = pd.DataFrame()
    lats = [lat - .01, lat, lat + .01]
    longs = [long - .01, long, long + .01]
    for l1 in lats:
        for l2 in longs:
          #if cities are too small (ie. Mobile, Alabama), this ignores locations w/out checkins
            try: 
                df = df.append(get_checkin_data(l1, l2),ignore_index = True)
            except: 
                pass 
    return df

In [4]:
#individual city data 
dataframe = get_checkins_around_latlong(37.7749,-122.4194)

dataframe

,checkins,engagement,id,name,overall_star_rating,single_line_address,website
0,168441,1459,683987574952302,"Mission District, San Francisco",NaN,"Mission District, San Francisco 94103, 94110",NaN
1,149273,29576,10150167656901809,Castro Theatre,4.7,"429 Castro St, San Francisco, California 94114",http://www.castrotheatre.com
2,90111,11779,10150177131215785,The Café - San Francisco,4.2,"2369 Market St, San Francisco, California 94114",http://www.cafesf.com
3,85535,34656,10150186494679894,Mission Dolores Park,4.6,"19th & Dolores St, San Francisco, California 9...",http://sfrecpark.org/destination/mission-dolor...
4,73198,45896,10150279241412907,Tartine Bakery,4.6,"600 Guerrero St, San Francisco, California 94110",http://www.tartinebakery.com https://twitter.c...
5,68140,5203,259774727457101,Hi Tops SF,4.3,"2247 Market St, San Francisco, California 94114",http://hitopssf.com
6,62316,12755,1453645444962405,Elbo Room,4.2,"647 Valencia St, San Francisco, California 94110",http://www.elbo.com
7,62186,21688,113634332122518,The Chapel,4.6,"777 Valencia St, San Francisco, California 94110",http://www.thechapelsf.com/ http://www.twitter...
8,58320,6062,10150105141374614,The Edge,4.3,"4149 18th St, San Francisco, California 94114",edgesf.com
9,56883,7988,334431973326200,Beaux,4.3,"2344 Market St, San Francisco, California 94114",http://beauxsf.com


In [5]:
#to get top 10 checkin locations:
#dataframe.to_csv('\\Users\\Olivia Steinkamp\\Documents\\SFdata.csv')

In [6]:
def get_city_data():
#from github - city lat/long   
    cities_data_url = "https://gist.githubusercontent.com/Miserlou/c5cd8364bf9b2420bb29/raw/2bf258763cdddd704f8ffd3ea9a3e81d25e2c6f6/cities.json"
    r = requests.get(cities_data_url)
    return r.json()

In [7]:

# figuring out what the # is for a given city
def get_number_for_city(city_name, state_name):
    i = 0
    for c in get_city_data():
        if (c['city'] == city_name) and (c['state'] == state_name):
            return i
        i += 1
    

        
def get_csv_of_checkin_data(n):
    
    df = pd.DataFrame()
    #pd.DataFrame(r.json())
    cities = get_city_data()
    cities = cities[n:n+52]   #...22 because info was lost???
    for city in cities:
        print('{}, {}'.format(city["city"],city['state']))
        print(n)
        try:
            city_df = get_checkins_around_latlong(city["latitude"],city["longitude"])
            city_df["city"] = "{}, {}".format(city["city"],city['state'])
            df = df.append(city_df,ignore_index = True)
            time.sleep(5)
            if len(city_df) > 0:
                print('...success!')
            else:
                print('...out of calls')
                break
        except:
            print('...cannot pull data for {}, {}'.format(city['city']))
            break
        n += 1

        try:
            df = df.drop_duplicates()
        except:
            pass

    time_string = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
    df.to_csv("checkin_data_MountVernon{}.csv").format(time_string)
    
    return df

In [8]:
get_csv_of_checkin_data(499)

Mount Vernon, New York
499
...success!
Fort Myers, Florida
500
...success!
Dothan, Alabama
501
...success!
Rancho Cordova, California
502
...success!
Redondo Beach, California
503
...success!
Jackson, Tennessee
504
...success!
Pasco, Washington
505
...success!
St. Charles, Missouri
506
...success!
Eau Claire, Wisconsin
507
...success!
North Richland Hills, Texas
508
...success!
Bismarck, North Dakota
509
...success!
Yorba Linda, California
510
...success!
Kenner, Louisiana
511
...success!
Walnut Creek, California
512
...success!
Frederick, Maryland
513
...success!
Oshkosh, Wisconsin
514
...success!
Pittsburg, California
515
...success!
Palo Alto, California
516
...success!
Bossier City, Louisiana
517
...success!
Portland, Maine
518
...success!
St. Cloud, Minnesota
519
...success!
Davis, California
520
...success!
South San Francisco, California
521
...success!
Camarillo, California
522
...success!
North Little Rock, Arkansas
523
...success!
Schenectady, New York
524
...success!
Gaither

/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


...out of calls


AttributeError: 'NoneType' object has no attribute 'format'

In [11]:
get_number_for_city("Decatur", "Illinois")

447

In [181]:
#df.to_csv("checkin_data_for_top_22.csv")  # remember to merge this with other checkin_data.csv

In [197]:
df = pd.read_csv("checkin_data_for_WashDC_to_SanBern.csv", encoding = "latin")
df2 = pd.read_csv("checkin_data_for_top_22.csv", encoding = "latin")
df = df.append(df2)
df = df[~df["single_line_address"].isnull()]

/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [173]:
#summary_df.to_csv("Checkins_and_UHS.csv")

In [26]:
#summary_df.to_csv('\\Users\\Olivia Steinkamp\\Documents\\Total_Checkins_Top10data.csv')

In [114]:
city_df = df[df["city"]=="New York"].sort_values("checkins", ascending = False)
total_checkins = city_df["checkins"].sum()

In [150]:
number_locations = len(city_df)
print(number_locations)

673


In [62]:
#save to .csv
#get_homogeneity_score.to_csv('\\Users\\Olivia Steinkamp\\Documents\\Total_Checkins_Top10data.csv')

NameError: name 'cvs' is not defined

In [258]:
#pd.DataFrame(get_city_data()).to_csv('city_data.csv')